In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [ ]:
# data -> dataframe with dataset
# attributes -> data.columns

In [11]:
# Discretize numerical variables

continuous_types = ["float64"]
numBins = 5

dict_attr = {}
dict_unique = {}

for key in data:
  dict_attr[key] = data[key].to_numpy()
  if str(dict_attr[key].dtype) in continuous_types:
    min = np.amin(dict_attr[key])
    max = np.amax(dict_attr[key])
    interval = (max - min) / numBins
    bins = np.arange(min, max, interval)
    dict_attr[key] = np.array([f"{key}-{elem}" for elem in np.digitize(dict_attr[key], bins).astype(str)])
  dict_unique[key] = np.unique(dict_attr[key])


In [12]:
# The different attributes are aggregated into a set variable
data_sets = np.apply_along_axis(set, 1, np.column_stack([dict_attr[key] for key in dict_attr.keys()]))

In [13]:
dict_unique = {}
for attribute in attributes:
  dict_unique[attribute] = attribute

Implementation

In [14]:
class Itemset:
  def __init__(self, attributes):
    self.attributes = dict.fromkeys(attributes, None)
    self.sons = []

  def add_item(self, key, item):
    self.attributes[key] = item

  def add_parent(self, parent_id):
    self.parent = parent_id

  def add_son(self, son_id):
    self.sons.append(son_id)

  def add_state(self, state):
    self.state = state

  def calc_support_rel(self, dataset):
    total_length = len(dataset)
    support_abs = 0

    for instance in dataset:
      if self.itemset.issubset(instance):
        support_abs += 1
    
    support_res = support_abs / total_length

    self.support = support_res

  def generate_itemset(self):
    self.itemset = set()
    for key in self.attributes:
      item = self.attributes[key]
      if item is not None:
        self.itemset.add(self.attributes[key])
    self.itemset = frozenset(self.itemset)
  
  def written_attributes(self):
    attr_list = []
    for key in self.attributes:
      if self.attributes[key] is not None:
        attr_list.append(key)
    return attr_list

  def create_son(self):
    created_son = deepcopy(self)
    created_son.itemset = None
    created_son.parent = None
    created_son.state = None
    created_son.support = None
    created_son.sons = []
    return created_son

  def copy_itemset(self):
    copied_itemset = deepcopy(self)
    return copied_itemset

  def define_support_state(self, min_support):
    if self.support >= min_support:
      self.state = "frequent"
    else:
      self.state = "no_frequent"
  
  def describe(self):
    print(f"Itemset: {self.itemset}\nAttributes: {self.attributes}\nSupport: {self.support}\nState: {self.state}\nParent: {self.parent}\nSons: {self.sons}\n")


In [26]:
min_support = 0.9

# Variable that represents the tree of itemsets
tree = {}
#Variable to controle the generation of duplicated itemsets
explored_itemsets = {}

maximals = []
closed = []

# Build the tree (base case)
available_level = 0
tree[available_level] = []
created_nodes = 0
for key in dict_unique:
  for elem in dict_unique[key]:
    itemset = Itemset(attributes)
    itemset.add_item(key, elem)
    itemset.add_parent(-1)
    itemset.generate_itemset()
    explored_itemsets[itemset.itemset] = created_nodes
    itemset.calc_support_rel(data_sets)
    itemset.define_support_state(min_support)
    tree[available_level].append(itemset)
    created_nodes += 1
    
available_level = 1


# Build the tree (general case)
for iter in range(1, len(attributes)):
  tree[available_level] = []
  created_nodes = 0
  # Itemsets of the previous level are checked to generate itemsets from the frequent itemsets
  for id, itemset in enumerate(tree[available_level-1]):
    if itemset.state == "frequent":
      for attribute in [attribute for attribute in attributes if attribute not in itemset.written_attributes()]:
        for item in dict_unique[attribute]:
          check_set = set(deepcopy(itemset.itemset))
          check_set.add(item)
          if frozenset(check_set) not in list(explored_itemsets.keys()):
            super_itemset = itemset.create_son()
            itemset.add_son(created_nodes)
            super_itemset.add_parent(id)
            super_itemset.add_item(attribute, item)
            super_itemset.generate_itemset()
            explored_itemsets[super_itemset.itemset] = created_nodes
            super_itemset.calc_support_rel(data_sets)
            super_itemset.define_support_state(min_support)
            tree[available_level].append(super_itemset)
            created_nodes += 1
          else:
            itemset.add_son(explored_itemsets[frozenset(check_set)])

  available_level += 1  

# Obtain maximal
for level in tree:
  for itemset in tree[level]:
    if itemset.state == "frequent":
      total_sons = len(itemset.sons)
      no_frequent_sons = 0
      for son in itemset.sons:
        if tree[level + 1][son].state == "no_frequent":
          no_frequent_sons += 1
      if total_sons == no_frequent_sons:
        maximals.append(set(itemset.itemset))

# Obtain closed
for level in tree:
  for itemset in tree[level]:
    isClosed = True
    if len(itemset.sons) == 0:
      isClosed = False
    else:
      for son in itemset.sons:
        if round(tree[level + 1][son].support, 4) == round(itemset.support, 4):
          isClosed = False
          break
    if isClosed == True:
      closed.append(set(itemset.itemset))
